In [4]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer
from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

conn = sqlite3.connect('../EconomicsFinal.db')

cursor = conn.cursor()
cursor.execute("SELECT title FROM EconomicsFinal")
rows = cursor.fetchall()

df = pd.DataFrame(rows, columns=['title'])

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

hdbscan_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

custom_stopwords = ['economics', 'economy', 'approach', 'new', 'study', 'models','based', 'model', 'paper','economic', 'time', 'results', 'show','using', 'used', 'also','data', 'network', 'one', 'two']

countvec = CountVectorizer(stop_words='english')

default_stopwords = countvec.get_stop_words()

stopwords = list(default_stopwords) + custom_stopwords

vectorizer_model = CountVectorizer(stop_words=stopwords,ngram_range=(2,3))

ctfidf_model = ClassTfidfTransformer()

representation_model = KeyBERTInspired()

topic_model = BERTopic(
  embedding_model=model,                    
  umap_model=umap_model,                    
  hdbscan_model=hdbscan_model,              
  vectorizer_model=vectorizer_model,        
  ctfidf_model=ctfidf_model,                
  representation_model=representation_model 
)

topic_model.fit(df['title'])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
topic_model.visualize_barchart(height=400,width=500)